# Forward Kinematics of the ABB robot

We would like to compute the forward kinematics of the ABB [IRB 910SC](https://new.abb.com/products/3HAC056161-001/irb-910sc) robot seen in class. 

<img src="./figs/ABB_robot.jpg" width="300">

From the [product specification](https://library.e.abb.com/public/9fc170040463477db86bda7065135457/3HAC056431%20PS%20IRB%20910SC-en.pdf?x-sign=Rsw7Ua3yCaT7/Evotwiwx6iTwfZgeP/val93/1o82csyGd1ejrv1gtswH4Bw8ppc) we can gather the following information:
1. The robot has 4 degrees of freedom
<img src="./figs/ABB_dofs.jpg" width="400">
2. The lengths of the different parts are:
<img src="./figs/ABB_lengths.jpg" width="800">

There are three possible versions of the robot with different sizes. We will choose the RB 910SC-3/0.55

From this information we can compute the forward kinematics of the robot. We do the following:
1. Fix a base frame - our world frame of reference
2. Choose frames attached on each link/body (first make sure you find all the links and joints connecting them)
3. Compute the relative transforms between each links
4. Compute the FK giving the pose of the end-effector in the base frame.

Steps 1 and 2 are illustrated on the picture below:
<img src="./figs/ABB_axes.jpg" width="400">

In [1]:
#setup nice plotting
%matplotlib widget

# we import useful libraries
import time
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True, precision=4)

## some basic transforms

We define below basic functions to get homogeneous transforms that correspond to pure translations and rotations around x,y or z axes

In [2]:
def translate(vector):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0:3,3] = vector

    ### we return the object
    return transform

def rotateX(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[1:3,1:3] = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])

    ### we return the object
    return transform

def rotateY(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0,0] = np.cos(angle)
    transform[0,2] = np.sin(angle)
    transform[2,0] = -np.sin(angle)
    transform[2,2] = np.cos(angle)

    ### we return the object
    return transform

def rotateZ(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0:2,0:2] = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])

    
    ### we return the object
    return transform

## Forward kinematics
Now we can compute the forward kinematics of the robot

In [3]:
def forward_kinematics_ABB(theta):
    # the relative pose of link 1 wrt the base is computed from
    # a translation and then rotation due to the motion of the first joint
    T_SL1 = translate([0,0,0.2577]) @ rotateZ(theta[0])
    
    # the relative pose of link 2 wrt to link 1 is
    # a translation and then rotation due to the motion of the second joint
    T_L1L2 = translate([0.3, 0., 0.]) @ rotateZ(theta[1])
    
    # the relative pose of link 3 wrt to link 2 is
    # a fixed translation followed by a translation due to joint 4 and a rotation due to joint 3
    T_L2L3 = translate([0.25, 0, theta[3]]) @ rotateZ(theta[2])
    
    # finally to get the relative pose from link 3 to the end effector
    # we need to move down and re-orient the frame to match the end-effector frame
    T_L3E = translate([0,0, 0.0402+0.180-.2577])  @ np.array([[0,0,1,0],[0,1,0,0],[-1,0,0,0],[0,0,0,1]])
    
    # the forward kinematics is then
    T_SF = T_SL1 @ T_L1L2 @ T_L2L3 @ T_L3E
    
    ### we return the pose of the end-effector
    return T_SF

In [4]:
# now we can compute the pose of the end-effector for various values of the joints
theta = np.array([0,0,0,0.])
T = forward_kinematics_ABB(theta)
print(f'for theta = {theta}\n\nthe pose of the end effector is\n\n {T}\n\n\n')


theta = np.array([np.pi/2,0,0,0.])
T = forward_kinematics_ABB(theta)
print(f'for theta = {theta}\n\nthe pose of the end effector is\n\n {T}\n\n\n')

theta = np.array([np.pi/2,np.pi/2,0.,0.5])
T = forward_kinematics_ABB(theta)
print(f'for theta = {theta}\n\nthe pose of the end effector is\n\n {T}\n\n\n')

theta = np.array([np.pi/2,np.pi/2,np.pi,0.5])
T = forward_kinematics_ABB(theta)
print(f'for theta = {theta}\n\nthe pose of the end effector is\n\n {T}\n\n\n')

for theta = [0. 0. 0. 0.]

the pose of the end effector is

 [[ 0.      0.      1.      0.55  ]
 [ 0.      1.      0.      0.    ]
 [-1.      0.      0.      0.2202]
 [ 0.      0.      0.      1.    ]]



for theta = [1.5708 0.     0.     0.    ]

the pose of the end effector is

 [[ 0.     -1.      0.      0.    ]
 [ 0.      0.      1.      0.55  ]
 [-1.      0.      0.      0.2202]
 [ 0.      0.      0.      1.    ]]



for theta = [1.5708 1.5708 0.     0.5   ]

the pose of the end effector is

 [[ 0.     -0.     -1.     -0.25  ]
 [ 0.     -1.      0.      0.3   ]
 [-1.      0.      0.      0.7202]
 [ 0.      0.      0.      1.    ]]



for theta = [1.5708 1.5708 3.1416 0.5   ]

the pose of the end effector is

 [[ 0.      0.      1.     -0.25  ]
 [ 0.      1.     -0.      0.3   ]
 [-1.      0.      0.      0.7202]
 [ 0.      0.      0.      1.    ]]





# Forward kinematics of the KUKA iiwa14 robot

Similarly we can compute the forward kinematics of the robot shown below, the KUKA iiwa14 robot which has 7 revolute joints. The function is shown in the next cell

<img src="./figs/kuka_robot.png" width="250">

In [5]:
def forward_kinematics_KUKA(theta):
    T_BJ1 = translate([0,0,0.1575]) @ rotateZ(theta[0])
    T_J1J2 = translate([0,0,0.2025]) @ rotateY(theta[1])
    T_J2J3 = translate([0,0,0.2045]) @ rotateZ(theta[2])
    T_J3J4 = translate([0,0,0.2155]) @ rotateY(-theta[3])
    T_J4J5 = translate([0,0,0.1845]) @ rotateZ(theta[4])
    T_J5J6 = translate([0,-0.0607, 0.2155]) @rotateY(theta[5])
    T_J6J7 = translate([0,0.0607, 0.0810])@ rotateZ(theta[6])
    T_J7E = translate([0,0,0.04])

    T_SF = T_BJ1 @ T_J1J2 @ T_J2J3 @ T_J3J4 @ T_J4J5 @ T_J5J6 @ T_J6J7 @ T_J7E
    
    ### we return the pose of the end-effector
    return T_SF

In [6]:
# now we can compute the pose of the end-effector for various values of the joints
theta = np.array([0.,0.,0.,0,0,0,0])
T = forward_kinematics_KUKA(theta)
print(f'for theta = {theta}\n\nthe pose of the end effector is\n\n {T}\n\n\n')

theta = np.array([np.pi/2,0.,0.,0,0,0,0])
T = forward_kinematics_KUKA(theta)
print(f'for theta = {theta}\n\nthe pose of the end effector is\n\n {T}\n\n\n')

theta = np.array([0.,0.5,0.,0,0,0,0])
T = forward_kinematics_KUKA(theta)
print(f'for theta = {theta}\n\nthe pose of the end effector is\n\n {T}\n\n\n')

theta = np.array([0.,0.5,1.,.6,0.6,0.3,0])
T = forward_kinematics_KUKA(theta)
print(f'for theta = {theta}\n\nthe pose of the end effector is\n\n {T}\n\n\n')

for theta = [0. 0. 0. 0. 0. 0. 0.]

the pose of the end effector is

 [[1.    0.    0.    0.   ]
 [0.    1.    0.    0.   ]
 [0.    0.    1.    1.301]
 [0.    0.    0.    1.   ]]



for theta = [1.5708 0.     0.     0.     0.     0.     0.    ]

the pose of the end effector is

 [[ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.     0.     1.     1.301]
 [ 0.     0.     0.     1.   ]]



for theta = [0.  0.5 0.  0.  0.  0.  0. ]

the pose of the end effector is

 [[ 0.8776  0.      0.4794  0.4511]
 [ 0.      1.      0.      0.    ]
 [-0.4794  0.      0.8776  1.1858]
 [ 0.      0.      0.      1.    ]]



for theta = [0.  0.5 1.  0.6 0.6 0.3 0. ]

the pose of the end effector is

 [[ 0.0858 -0.9833  0.1605  0.272 ]
 [ 0.9795  0.0538 -0.1944 -0.2136]
 [ 0.1825  0.1739  0.9677  1.1939]
 [ 0.      0.      0.      1.    ]]



